In [1]:
!pip install requests beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random  # 新增隨機數，讓行為更像真人

# --- 設定區 ---
PTT_URL = "https://www.ptt.cc"
BOARD = "Beauty"
TARGET_URL = f"{PTT_URL}/bbs/{BOARD}/index.html"

# 建立一個 Session (像是一個持續的瀏覽器視窗)
rs = requests.Session()
rs.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Cookie": "over18=1"
})

# --- 函數區 ---
def get_soup(url, retries=3):
    """
    增加重試機制：如果失敗，會等待後再試一次
    """
    for i in range(retries):
        try:
            resp = rs.get(url, timeout=10) # 設定超時限制
            if resp.status_code == 200:
                return BeautifulSoup(resp.text, "html.parser")
            else:
                print(f"⚠️ 網頁回應異常: {resp.status_code}")
        except Exception as e:
            print(f"⚠️ 連線失敗 (嘗試 {i+1}/{retries}): {e}")
            time.sleep(3) # 失敗時休息 3 秒再試
            
    print("❌ 無法取得網頁，跳過此連結")
    return None

def save_image(img_url, title):
    folder_name = "downloaded_images/" + title.replace("/", "_").replace(":", "_").replace("?", "")
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    try:
        # 下載圖片也要慢一點
        time.sleep(random.uniform(0.5, 1.5)) 
        
        img_resp = rs.get(img_url, stream=True, timeout=10)
        if img_resp.status_code == 200:
            file_name = img_url.split("/")[-1]
            file_path = f"{folder_name}/{file_name}"
            
            with open(file_path, "wb") as f:
                for chunk in img_resp.iter_content(1024):
                    f.write(chunk)
            print(f"   ✅ [下載成功] {file_name}")
        else:
            print(f"   ⚠️ [圖片失效] {img_url}")
    except Exception as e:
        print(f"   ❌ [下載錯誤] {e}")

def parse_article(article_url, title):
    soup = get_soup(article_url)
    if not soup: return
    
    main_content = soup.find(id="main-content")
    if not main_content: return
    
    links = main_content.find_all("a")
    img_extensions = ('.jpg', '.jpeg', '.png', '.gif')
    
    found = False
    for link in links:
        href = link.get("href")
        if href and href.lower().endswith(img_extensions):
            found = True
            save_image(href, title)
            
    if not found:
        print("   (此文章無圖片)")

# --- 主執行區 ---
print("🚀 PTT 爬蟲啟動中 (溫和版)...")
current_url = TARGET_URL
pages_to_crawl = 1 

for i in range(pages_to_crawl):
    print(f"\n📂 正在讀取第 {i+1} 頁...")
    soup = get_soup(current_url)
    if not soup: 
        print("無法讀取列表頁，程式終止。")
        break
        
    articles = soup.find_all("div", class_="r-ent")
    for article in articles:
        title_div = article.find("div", class_="title")
        if not title_div.a: continue
        
        title = title_div.a.text
        link = PTT_URL + title_div.a["href"]
        
        if "公告" in title: continue
            
        print(f"正在處理：{title}")
        parse_article(link, title)
        
        # 每處理完一篇文章，休息 1~3 秒 (隨機)
        # 這是避免被鎖的關鍵！
        wait_time = random.uniform(1, 3)
        print(f"   ☕ 休息 {wait_time:.1f} 秒...")
        time.sleep(wait_time)
        
    controls = soup.find("div", class_="btn-group-paging")
    try:
        prev_link = controls.find_all("a")[1]["href"]
        current_url = PTT_URL + prev_link
    except:
        print("找不到上一頁按鈕，可能已到盡頭。")
        break

print("\n🎉 任務完成！")

🚀 PTT 爬蟲啟動中 (溫和版)...

📂 正在讀取第 1 頁...
正在處理：[正妹] 可愛小雀斑
   ✅ [下載成功] W8DNz4rQ_o.jpg
   ✅ [下載成功] 2Nc0gZez_o.jpg
   ✅ [下載成功] hwt5R5Mi_o.jpg
   ✅ [下載成功] 3bQfpHJm_o.jpg
   ✅ [下載成功] RQu9Cb6P_o.jpg
   ✅ [下載成功] qSXbU2W0_o.jpg
   ☕ 休息 2.8 秒...
正在處理：[正妹] estelle.belmont,愛分享健身的女孩
   ✅ [下載成功] wihDlpCn_o.jpg
   ✅ [下載成功] 2g6TcW0q_o.jpg
   ✅ [下載成功] OermNaW4_o.gif
   ✅ [下載成功] wkbNjAdc_o.jpg
   ❌ [下載錯誤] HTTPSConnectionPool(host='images2.imgbox.com', port=443): Read timed out.
   ✅ [下載成功] UKbb2s1i_o.jpg
   ✅ [下載成功] Kf9AmEKU_o.jpg
   ✅ [下載成功] ySG7JObr_o.gif
   ❌ [下載錯誤] HTTPSConnectionPool(host='images2.imgbox.com', port=443): Read timed out.
   ✅ [下載成功] 1jAsK9AB_o.jpg
   ✅ [下載成功] bHmFir0a_o.jpg
   ☕ 休息 1.4 秒...
正在處理：[正妹] Cosplay 2452 日本 艾蜜莉亞
   ✅ [下載成功] DfEXo.jpg
   ✅ [下載成功] VGsL7.jpg
   ✅ [下載成功] 8d9Tx.jpg
   ✅ [下載成功] 89IE3.jpg
   ✅ [下載成功] Nfuk1.jpg
   ✅ [下載成功] cgD2r.jpg
   ✅ [下載成功] HXloy.jpg
   ✅ [下載成功] e8dN5uA.png
   ☕ 休息 1.1 秒...
正在處理：[正妹] 在台北的曹薇娟 三張
   ✅ [下載成功] jIc6lA.jpg
   ✅ [下載成功] 4GHPmo.jpg
   ✅ [下載

KeyboardInterrupt: 